### MMLU

Até agora tratamos de métodos criados por volta de 2005. Vamos avançar agora uns 15 anos para frente e avaliar um método que foi criado para avaliar os modelos de linguagens iniciais. 

MMLU significa Massive Multitask Language Understanding e trata se de um benchmark criado para avaliar a capacidade das LLMs em áreas diversas de conhecimento. Consiste de aproximadamente 16 mill questões variadas do tipo multipla escolha, disponibilizadas por meio de um dataset que pode ser baixado do link https://people.eecs.berkeley.edu/~hendrycks/data.tar

Ao abrir o arquivo compactado, o leitor perceberá que o dataset está dividido em diversos arquivos .csv, organizados por assunto, o que permite escolher o domínio de assunto a ser avaliado.

O repositório original foi criado por Dan Hendrycks e alguns pesquisadores e pode ser consultado no link https://github.com/hendrycks/test
Não deixem de consultar também o paper original: https://arxiv.org/pdf/2009.03300
Cada teste consiste de uma pergunta, as opções de respostas disponíveis e um gabarito. A idéia então é submeter a pergunta junto com as opções de resposta para que a LLM tente responder. Depois essa resposta deve ser comparada contra o gabarito. As respostas são informadas sempre como opções ‘a’,’b’,’c’, etc
O MMLU original foi criado no início de 2021, e muita coisa tem evoluído desde então. Natural que com a evolução dos modelos, eles alcancem scores cada vez maiores neste teste, mas além disso, existe uma preocupação de que os modelos sendo avaliados se utilizem de ferramentas como WebSearch na resposta, sem contar a possibilidade de que o teste pode ser vazado e consumido durante o treinamento de novos modelos de linguagem, o que acaba por contaminar a resposta e o score obtido.

Por conta disso, métodos derivados tem sido criados, buscando inibir a possibilidade de utilizar estes testes no treinamento do modelo ou ao menos dificultar este processo. Caberia um livro aqui, mas podemos citar por exemplo alguns substitutos como AGIEval, GPQA, MMLU-Pro. 
Vamos exercitar o processo de avaliação com MMLU, através do notebook abaixo. Para este exercício vou utilizar 2 serviços do Oracle OCI: OCI Generative AI (para consumir um modelo de LLM hospedado na nuvem Oracle que no caso será o Open AI GPT 4.1 nano) e o serviço OCI Data Science, comentado no início do artigo , para criação do Jupyter Notebook.



In [115]:
#! pip install langchain_community
#! pip install oci

In [116]:
import oci

#Bibliotecas para uso do langchain + OCI Generative AI
from langchain_community.chat_models import ChatOCIGenAI
from langchain_community.chat_models import oci_generative_ai


### Autenticação OCI

In [117]:
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaacuafyhpnjnsp5luoo3dqklsdi2ysobswq3irzu664gl3cjhvcjpa"
AUTH_TYPE = "API_KEY" # The authentication type to use, e.g., API_KEY (default), SECURITY_TOKEN, INSTANCE_PRINCIPAL, RESOURCE_PRINCIPAL.

# Auth Config
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)

### Instância OCI Generative AI com modelo gpt 4.1 nano, executado dentro do OCI

In [118]:
# Service endpoint
#Atenção ao endpoint, porque alguns modelos estão disponíveis em regiões especificas. 
endpoint = "https://inference.generativeai.us-ashburn-1.oci.oraclecloud.com"

In [119]:
llm_oci = ChatOCIGenAI(
    model_id="ocid1.generativeaimodel.oc1.iad.amaaaaaask7dceyacxqiaijwxalbynhst6oyg4wttzagz3dai4y2rnwh6wrq", #open ai gpt 4.1 nano
    service_endpoint=endpoint,
    compartment_id=COMPARTMENT_ID,
    provider="meta",
    model_kwargs={
        "temperature": 0.1,
        "max_tokens": 150,
        #"frequency_penalty": 0,
        #"presence_penalty": 0,
        "top_p": 0.75,
        "top_k": None#
        #"seed": None
    },
    auth_type=AUTH_TYPE,
    auth_profile="DEFAULT",
    auth_file_location='config'
)

### Teste 1

Este é um teste bem simples, para exemplificar o método de avaliação. Caso queira rodar contra um dataset do mmlu, pode executar o teste 2

In [120]:
mmlu_dataset = [
    {
        "question": "Qual destes minerais é necessário à formação de hemoglobina?",
        "choices": ["a) Ferro", "b) Cálcio", "c) Potássio", "d) Sódio"],
        "answer": "a"
    },
    {
        "question": "Quem escreveu 'Dom Casmurro'?",
        "choices": ["a) Machado de Assis", "b) Clarice Lispector", "c) José de Alencar", "d) Monteiro Lobato"],
        "answer": "a"
    }
]

In [121]:

def perguntar_ao_modelo_oci(llm, question, choices):
    prompt = question + "\n"
    for opt in choices:
        prompt += opt + "\n"
    prompt += "Responda apenas com a letra da alternativa correta (a, b, c ou d):"

    response = llm.invoke([prompt])
    resposta = response.content.strip().lower()
    print(resposta)
    # Captura apenas 'a', 'b', 'c', ou 'd'
    for alt in ['a', 'b', 'c', 'd']:
        if resposta.startswith(alt):
            return alt
    #fallback se o modelo retornar algo inesperado
    return "-"


In [122]:

corretos = 0
for item in mmlu_dataset:
    resposta_modelo = perguntar_ao_modelo_oci(llm_oci, item['question'], item['choices'])
    print(f"Pergunta: {item['question']}")
    print(f"Resposta do modelo: {resposta_modelo} | Resposta correta: {item['answer']}\n")
    if resposta_modelo == item['answer']:
        corretos += 1

acuracia = corretos / len(mmlu_dataset)
print(f"Acurácia final do modelo: {acuracia:.2%}")

a
Pergunta: Qual destes minerais é necessário à formação de hemoglobina?
Resposta do modelo: a | Resposta correta: a

a
Pergunta: Quem escreveu 'Dom Casmurro'?
Resposta do modelo: a | Resposta correta: a

Acurácia final do modelo: 100.00%


### Teste 2

Para o segundo teste, já utilizando o dataset do mmlu, escolhi apenas um dos arquivos csv por uma questão de praticidade. O tema escolhido foi astronomia (amo desde criança).

Armazenei junto ao notebook uma pasta chamada mmlu_dataset onde tenho todos os datasets organizados por tema.

In [123]:
import csv

mmlu_dataset = []

with open('mmlu_dataset/astronomy_test.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        mmlu_dataset.append({
            "question": row[0],
            "choices": [
                f"a) {row[1]}",
                f"b) {row[2]}",
                f"c) {row[3]}",
                f"d) {row[4]}"
            ],
            "answer": row[5].strip().lower()
        })


def perguntar_ao_modelo_oci(llm, question, choices):
    prompt = question + "\n"
    for opt in choices:
        prompt += opt + "\n"
    prompt += "Responda apenas com a letra da alternativa correta (a, b, c ou d):"
    response = llm.invoke([prompt])
    resposta = response.content.strip().lower()
    #print(resposta)
    for alt in ['a', 'b', 'c', 'd']:
        if resposta.startswith(alt):
            return alt
    return "-"

def rodar_teste_mmlu_memoria(llm, mmlu_dataset):
    resultados = []
    for item in mmlu_dataset:
        question = item["question"]
        choices = item["choices"]
        resposta_correta = item["answer"].lower()
        resposta_modelo = perguntar_ao_modelo_oci(llm, question, choices)
        resultados.append({
            'question': question,
            'resposta_modelo': resposta_modelo,
            'resposta_correta': resposta_correta,
            'acerto': resposta_modelo == resposta_correta
        })
    return resultados

resultados = rodar_teste_mmlu_memoria(llm_oci, mmlu_dataset)
acertos = sum([1 for r in resultados if r['acerto']])
print(f"Acurácia: {acertos/len(resultados):.2%}")

Acurácia: 84.21%
